##                                                  **Cifra Vigenère Parte 1: Codificação e Decodificação** ##


In [5]:
import unidecode

ALFABETO = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
CARACTERES_PERMITIDOS = " .,'"


def normalizar_txt(texto):
    texto = unidecode(texto)
    txt_processado = ""
    for char in texto.upper():
        if char in ALFABETO:
            txt_processado += char

        if char in CARACTERES_PERMITIDOS:
            txt_processado += char

    return txt_processado

def extender_chave(chave, tamanho_texto):  # Normaliza a chave também
    chave_normalizada = ""
    for char in chave.upper():
        if char in ALFABETO:
            chave_normalizada += char

    if not chave_normalizada:
        return "Chave Inválida"

    return (chave_normalizada * (tamanho_texto // len(chave_normalizada) + 1))[
        :tamanho_texto
    ]

In [6]:
def criptografar(chave, texto):  #  chave e plaintext para o Vigenere

    chave_normalizada = extender_chave(chave, len(texto))
    plaintext_normalizado = normalizar_txt(texto)
    texto_cifrado = []
    indice = 0

    for char_do_texto in plaintext_normalizado:
        
        if char_do_texto in ALFABETO:

            valor_char_texto = ALFABETO.find(char_do_texto)
            valor_char_chave = ALFABETO.find(chave_normalizada[indice])

            valor_char_cifra = (valor_char_texto + valor_char_chave) % 26
            texto_cifrado.append(ALFABETO[valor_char_cifra])
            indice += 1
        else:
            texto_cifrado.append(char_do_texto)

    return "".join(texto_cifrado)


def decifrar(chave, texto_cifrado):
    texto_original = []
    indice = 0

    chave_normalizada = extender_chave(chave, len(texto_cifrado))
    texto_cifrado = normalizar_txt(texto_cifrado)

    for char_da_cifra in texto_cifrado:

        if char_da_cifra in ALFABETO:

            valor_char_cifra = ALFABETO.find(char_da_cifra)
            valor_char_chave = ALFABETO.find(chave_normalizada[indice])

            valor_char_texto = (valor_char_cifra - valor_char_chave + 26) % 26
            texto_original.append(ALFABETO[valor_char_texto])
            indice += 1

        else:
            texto_original.append(char_da_cifra)

    return "".join(texto_original)

##                                                  **Cifra Vigenère Parte 2: Ataque de Frequências** ##


1.  Calcular a frequência das letras.
2.  Filtrar o texto para o alfabeto
3.  Dividir o texto cifrado em cosets baseados no tamanho da chave
4.  Calcular o Índice de Coincidência IC
5.  Encontrar a letra chave para um coset usando método de correlação

In [ ]:
from source import obter_apenas_letras
import collections

frequencia_portugues = {
    "A": 0.1463,"B": 0.0104,"C": 0.0388,"D": 0.0499,"E": 0.1257,"F": 0.0102,"G": 0.0130,"H": 0.0078,"I": 0.0618,"J": 0.0040,"K": 0.0002,"L": 0.0278,"M": 0.0474,
    "N": 0.0505,"O": 0.1073,"P": 0.0252,"Q": 0.0120,"R": 0.0653,"S": 0.0680,"T": 0.0434,"U": 0.0368,"V": 0.0158,"W": 0.0004,"X": 0.0021,"Y": 0.0001,"Z": 0.0047,
}

frequencia_ingles = {
    "A": 0.08167,"B": 0.01492,"C": 0.02782, "D": 0.04253,"E": 0.12702,"F": 0.02228,"G": 0.02015, "H": 0.06094,"I": 0.06966,"J": 0.00153,"K": 0.00772,"L": 0.04025,"M": 0.02406,
    "N": 0.06749,"O": 0.07507,"P": 0.01929,"Q": 0.00095,"R": 0.05987,"S": 0.06327,"T": 0.09056,"U": 0.02758,"V": 0.00978,"W": 0.02360,"X": 0.00150,"Y": 0.01974,"Z": 0.00074,
}


def normalizar_texto_para_analise_frequencia(texto):
    texto_processado_passo1 = unidecode(texto)
    return obter_apenas_letras(texto_processado_passo1)


def calcular_frequencia_letras(texto_filtrado):
    if not texto_filtrado:
        return {letra: 0.0 for letra in ALFABETO}
        
    contagem = collections.Counter(texto_filtrado)
    total_letras = len(texto_filtrado)
    frequencias = {letra: contagem.get(letra, 0) / total_letras
                                                                 for letra in ALFABETO}
    return frequencias


def calcular_contagens_letras(texto_apenas_letras):
    contagens = {char: 0 for char in ALFABETO}
    comprimento_texto = 0
    for char in texto_apenas_letras:
        if char in contagens:
            contagens[char] += 1
            comprimento_texto += 1
    return contagens, comprimento_texto


IC_ESPERADO_PORTUGUES = sum(p**2 for p in frequencia_portugues.values())
IC_ESPERADO_INGLES = sum(p**2 for p in frequencia_ingles.values())

In [ ]:
import source.funcoes as funcoes

def obter_subtexto_coluna( # 'coset'
    texto_apenas_letras, tamanho_chave , indice_coluna
):
    subtexto = ""
    for i in range(indice_coluna, len(texto_apenas_letras), tamanho_chave):
        subtexto += texto_apenas_letras[i]
    return subtexto


def calcular_ic_texto(texto_apenas_letras):

    if not texto_apenas_letras:
        return 0.0
        
    contagens, comprimento_texto = funcoes.calcular_contagens_letras(
        texto_apenas_letras
    )

    if comprimento_texto < 2:
        return 0.0

    soma_numerador_ic = 0.0
    for letra_idx in funcoes.ALFABETO:
        ni = contagens.get(letra_idx, 0)
        soma_numerador_ic += ni * (ni - 1)

    denominador_ic = comprimento_texto * (comprimento_texto - 1)
    if denominador_ic == 0:
        return 0.0

    ic = soma_numerador_ic / denominador_ic
    return ic

In [ ]:
def encontrar_tamanho_chave(texto_cifrado_apenas_letras,tamanho_min_chave, tamanho_max_chave , idioma_alvo = "pt"):

    ic_alvo_lingua = funcoes.IC_ESPERADO_PORTUGUES
    #nome_idioma_print = "Português"

    #if idioma_alvo.lower() == "en":
        #if hasattr(funcoes, "IC_ESPERADO_INGLES"):
            #ic_alvo_lingua = funcoes.IC_ESPERADO_INGLES
            #nome_idioma_print = "Inglês"
        #else:
           # print(
               # f"Aviso: Constante IC_ESPERADO_INGLES não encontrada em funcoes.py. Usando Português como fallback para IC."
           # )
    #elif idioma_alvo.lower() != "pt":
       # print(
           # f"Aviso: Idioma alvo '{idioma_alvo}' não reconhecido para IC. Usando Português como fallback para IC."
       # )

    melhor_tamanho_chave = 0
    menor_diferenca_abs_ic = float("inf")

    #print(
        #f"\nProcurando tamanho da chave (IC alvo para {nome_idioma_print}: {ic_alvo_lingua:.4f}):"
    #)

    for tamanho_candidato in range(tamanho_min_chave, tamanho_max_chave + 1):
        if tamanho_candidato <= 0:
            continue

        ics_das_colunas = []
        for i in range(tamanho_candidato):
            coluna_texto = obter_subtexto_coluna(
                texto_cifrado_apenas_letras, tamanho_candidato, i
            )
            if coluna_texto:
                ics_das_colunas.append(calcular_ic_texto(coluna_texto))

        if not ics_das_colunas:
            ic_medio_colunas = 0.0
        else:
            ic_medio_colunas = sum(ics_das_colunas) / len(ics_das_colunas)

        diferenca_atual = abs(ic_medio_colunas - ic_alvo_lingua)

       # print(
            #f"  Tamanho candidato {tamanho_candidato:2d}: IC Médio das Colunas = {ic_medio_colunas:.4f}, (Diferença para IC alvo: {diferenca_atual:.4f})"
        #)

        if diferenca_atual < menor_diferenca_abs_ic:
            menor_diferenca_abs_ic = diferenca_atual
            melhor_tamanho_chave = tamanho_candidato

    if melhor_tamanho_chave == 0 and tamanho_min_chave > 0:
       # print(
           # f"Aviso: Não foi possível determinar um tamanho de chave ótimo. Usando tamanho mínimo: {tamanho_min_chave}"
       # )
        melhor_tamanho_chave = tamanho_min_chave

    #print(f"Melhor estimativa para tamanho da chave: {melhor_tamanho_chave}")
    return melhor_tamanho_chave

def encontrar_char_chave_para_coluna( coluna_apenas_letras, idioma_alvo = "pt"):

    #if not coluna_apenas_letras:
        #return "?"

    frequencias_lingua_alvo = funcoes.frequencia_portugues

    #if idioma_alvo.lower() == "en":
        #if hasattr(funcoes, "frequencia_ingles"):
           #frequencias_lingua_alvo = funcoes.frequencia_ingles
       # else:
           # print(
           #     f"Aviso: frequencia_ingles não encontrada em funcoes.py. Usando Português para encontrar char da chave."
           # )
    #elif idioma_alvo.lower() != "pt":
        #print(
        #    f"Aviso: Idioma '{idioma_alvo}' não reconhecido para frequências. Usando Português para encontrar char da chave."
        #)

    frequencias_esperadas_vetor = [
        frequencias_lingua_alvo.get(char, 0.0) for char in funcoes.ALFABETO
    ]
    melhor_char_chave = "A"
    maior_correlacao = -1.0

    for i_chave_candidata in range(len(funcoes.ALFABETO)):
        char_chave_candidato = funcoes.ALFABETO[i_chave_candidata]
        coluna_decifrada_tentativa_lista = []

        for char_cifrado_coluna in coluna_apenas_letras:
            valor_char_cifrado = funcoes.ALFABETO.find(char_cifrado_coluna)
            valor_char_chave_cand = i_chave_candidata

            valor_char_decifrado = (
                valor_char_cifrado - valor_char_chave_cand + len(funcoes.ALFABETO)
            ) % len(funcoes.ALFABETO)
            coluna_decifrada_tentativa_lista.append(
                funcoes.ALFABETO[valor_char_decifrado]
            )

        coluna_decifrada_str = "".join(coluna_decifrada_tentativa_lista)
        frequencias_observadas_tentativa = funcoes.calcular_frequencias_observadas(
            coluna_decifrada_str
        )
        frequencias_observadas_vetor = [
            frequencias_observadas_tentativa.get(char, 0.0) for char in funcoes.ALFABETO
        ]

        correlacao_atual = sum(
            p_esp * q_obs
            for p_esp, q_obs in zip(
                frequencias_esperadas_vetor, frequencias_observadas_vetor
            )
        )

        if correlacao_atual > maior_correlacao:
            maior_correlacao = correlacao_atual
            melhor_char_chave = char_chave_candidato

    return melhor_char_chave



def recuperar_senha( texto_cifrado_apenas_letras , tamanho_chave , idioma_alvo ):
    if tamanho_chave <= 0 or not texto_cifrado_apenas_letras:
        return ""

    senha_recuperada_lista = []
    #print(
    #    f"\nRecuperando caracteres da chave (tamanho {tamanho_chave}, idioma {idioma_alvo.upper()}):"
    #)

    for i in range(tamanho_chave):
        coluna_atual = obter_subtexto_coluna(
            texto_cifrado_apenas_letras, tamanho_chave, i
        )
        #if not coluna_atual:
            #print(f"  Coluna {i+1} está vazia. Adicionando '?' à chave.")
            #senha_recuperada_lista.append("?")
            #continue

        char_chave_encontrado = encontrar_char_chave_para_coluna(
            coluna_atual, idioma_alvo
        )
        senha_recuperada_lista.append(char_chave_encontrado)
        #print(f"  Coluna {i+1}: Caractere da chave = {char_chave_encontrado}")

    return "".join(senha_recuperada_lista)